In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from google.colab import files

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Redes Neurais

Hiperparâmetros:

- loss: função de perda para avaliação do erro. ex: mean_squared_error
- optimizer: otimizador de ajuste de pesos da rede. stochastic gradient descnet
- metrics: a métrica que a rede vai utilizar para avaliar a performance


In [22]:
spam = pd.read_csv("spam.csv")

In [23]:
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [24]:
encoder = LabelEncoder()
y = encoder.fit_transform(spam['Category'])
print(y)

[0 0 1 ... 0 0 0]


In [25]:
X = spam['Message'].values
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [27]:
print(X_train.shape, X_test.shape)

(3900,) (1672,)


Preparando os dados para a criação do embedding

transformando os dados



In [28]:
token = Tokenizer(num_words=1000)

In [29]:
token.fit_on_texts(X_train)

In [30]:
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [31]:
print(len(X_train), len(X_test))

3900 1672


In [33]:
print(X_train)

[[387, 245, 325, 450, 917, 431, 1, 169], [19, 4, 112, 93, 6, 40, 358], [49, 135, 90, 7, 473, 90, 204, 499, 632, 5, 918], [412, 413, 3, 298, 535, 13, 414, 919], [91, 128, 75, 19, 14, 9, 160, 782, 293], [102, 1, 231, 83, 70, 8, 56, 118, 415, 26, 24, 11, 536, 2, 42, 12, 4, 633], [214, 136, 22, 155, 241, 920], [21, 215, 29, 54, 783, 79, 106, 3, 568, 451, 1, 2, 178], [14, 9, 61, 474, 537, 537], [216, 160, 205, 29, 318, 19], [98, 5, 152, 1, 60, 35, 6, 9, 5, 152, 41, 9, 5, 358, 10, 34, 1, 134, 1, 53, 6, 40, 5, 849, 15, 134, 6, 197, 569, 60], [99, 9, 66, 6, 10, 72, 601, 687, 37, 921, 452], [30, 6, 369, 40, 10, 8, 88, 1, 55, 305, 538, 319, 281], [48], [12, 115, 570, 453, 688, 17, 3, 108, 39, 30, 91, 260, 16, 46, 784, 26, 67, 87], [37, 32, 23, 359, 32, 8, 432, 24, 36, 39, 39, 49, 920], [32, 48, 475, 14, 201, 1, 56, 18, 288, 97, 282, 25, 1, 6, 271, 56, 850, 201, 6, 157, 23, 137], [117], [785, 726, 40, 34], [23, 50, 41, 20, 3, 8, 5], [37, 32, 54, 12, 5, 516, 9, 14, 48, 36, 1, 261, 8, 118], [6, 17,

In [34]:
X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

In [35]:
print(X_train)

[[387 245 325 ...   0   0   0]
 [ 19   4 112 ...   0   0   0]
 [ 49 135  90 ...   0   0   0]
 ...
 [ 32 956 721 ...   0   0   0]
 [  1 243   0 ...   0   0   0]
 [120  73  38 ...   0   0   0]]


# Implementação da rede neural para criação do Embedding

In [36]:
print(len(token.word_index))

7500


In [39]:
model = Sequential()
model.add(Embedding(input_dim=len(token.word_index),
                    output_dim=50,
                    input_length=500))
model.add(Flatten())

model.add(Dense(units=10, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation="sigmoid")) #camada de saída

In [40]:
model.compile(loss='mean_squared_error',
              optimizer="adam",
              metrics=["accuracy", "precision"])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.fit(X_train, y_train,
          #quantidade de vezes que os dados passarão pela rede(muito alto, vai levar muito tempo, muito baixo, o modelo pode não aprender)
          epochs=30,
          batch_size=10, #a cada quantos registros a rede será atualizada
          validation_data=(X_test, y_test), #passa também os dados de teste para a rede neural ir corrigindo
          verbose=True
          )

Epoch 1/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.8590 - loss: 0.1077 - precision: 0.3176 - val_accuracy: 0.9683 - val_loss: 0.0435 - val_precision: 1.0000
Epoch 2/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9674 - loss: 0.0456 - precision: 0.9048 - val_accuracy: 0.9839 - val_loss: 0.0335 - val_precision: 0.9668
Epoch 3/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9763 - loss: 0.0363 - precision: 0.9107 - val_accuracy: 0.9850 - val_loss: 0.0282 - val_precision: 0.9854
Epoch 4/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9773 - loss: 0.0299 - precision: 0.8963 - val_accuracy: 0.9868 - val_loss: 0.0245 - val_precision: 0.9951
Epoch 5/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9833 - loss: 0.0244 - precision: 0.9346 - val_accuracy: 0.9850 - val_loss: 0.0221 - val_precision: 0.9950
Epoch 6/30
390/390 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9835 - loss: 0.0223 - precision: 0.9268 - val_accuracy: 0.9862 - val_loss:

In [42]:
loss, accuracy, precision = model.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
print("Precision: ", precision)

53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9867 - loss: 0.0125 - precision: 0.9811
Loss:  0.012053752318024635
Accuracy:  0.9868420958518982
Precision:  0.9902912378311157


In [43]:
prevs = model.predict(X_test)
prevs

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[5.6081367e-26],
       [6.1746181e-28],
       [1.0818049e-33],
       ...,
       [2.5678636e-28],
       [1.0000000e+00],
       [2.1695435e-38]], dtype=float32)

In [44]:
#se for True é spam
prevs = prevs > 0.5
prevs

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [45]:
cm = confusion_matrix(y_test, prevs)
print(cm)

[[1446    2]
 [  20  204]]
